# Introdcution
Please upvote👍 if you find this  notebook is useful!
This is the introduction notebook for **FFT**.  
I applyed FFT analysis to time series cryptocurrency data.  

There are some NAN value in data, which makes FFT does not work.  
It is not good, but I tried naive approch which drop the row which has NAN value.  


Also, I tried **Lomb-Scargle periodogram** which can treat those type of data.
(But this part is pending...)

[💰Wavelet x AI x Stock💰](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/291909)is related discussion I prepared.

# Check dataset

In [ ]:
import pandas as pd

In [ ]:
asset_df = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset_df.info()
asset_df

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sorted_asset_df = asset_df.sort_values('Weight')

x = np.array(sorted_asset_df["Asset_Name"].values)
y = np.array(sorted_asset_df["Weight"].values)

plt.figure(figsize=(12,8))
plt.bar(x, y, align="center")
plt.xticks(rotation=70,fontsize=15)
plt.title("Weight of cryptocurrency",fontsize=30)
plt.xlabel("Asset Name",fontsize=20)
plt.ylabel("Weight",fontsize=20)
plt.show()

In [ ]:
train_df = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
train_df.info()
train_df.head()

# Time series plot

In [ ]:
def plot_time_series(asset_id:int, asset_value:str, start:int = 0, end:int = len(train_df), visualise:bool = True,drop_na = True):
    """Plot time series data from train_df(train.csv)

    Args:
        asset_id (int): Asset_ID.
        asset_value (str): Count, Open, High, Low, Close, Volume, VWAP, Target.
        start(int): start index
        end(int): end index
        visualise(bool): If true, the gragh is plot
        drop_na(bool):
    """
    
    asset_name = asset_df[asset_df["Asset_ID"]==asset_id]["Asset_Name"].values[0]
    train_crypro_df = train_df[train_df["Asset_ID"]==asset_id].reset_index(drop=True)
    
    if drop_na:
        train_crypro_df = train_crypro_df.dropna(subset=[asset_value])
    
    train_crypro_slice_df = train_crypro_df.iloc[start:end]
    #train_crypro_df.info()
    #train_crypro_df.head()
    t = np.array(list(train_crypro_slice_df.index))
    y = np.array(train_crypro_slice_df[asset_value].values)

    if visualise:
        plt.figure(figsize=(12,8))
        plt.plot(t, y)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.title(f"{asset_value} of {asset_name}\n(start={start},end={end})",fontsize=30)
        plt.xlabel("Time",fontsize=20)
        plt.ylabel(f"{asset_value}",fontsize=20)
        plt.show()
    
    
    return t, y

In [ ]:
t, y = plot_time_series(asset_id = 8,asset_value = "Target",visualise=True)

In [ ]:
t,y = plot_time_series(asset_id = 8,asset_value = "Target",start = 0,end = 1000)

In [ ]:
t,y = plot_time_series(asset_id = 8,asset_value = "Close")

In [ ]:
t,y = plot_time_series(asset_id = 8,asset_value = "Close",start = 0,end = 1000)

# FFT

## Example1: sin curve

In [ ]:
def plot_sin_curve(a:float, f:float, pd:float, end_time:float, sample_num:int,):
    """Generate sin curve and plot

    Args:
        a (float): Amplitude.
        f (float): Frequency[Hz].
        pd(float):phase difference(rad).
        end_time (float): End time(sec).
        sample_num(int) sample number.
    
    Returns:
        t(np.ndarray): numpy array of time
        y(np.ndarray):numpy array of y
    """
    t = np.linspace(0, end_time, sample_num)
    y = a * np.sin(2*np.pi*f*t + pd)
    plt.figure(figsize=(12,8))
    plt.plot(t,y)
    plt.grid()
    plt.show()
    
    return t,y

In [ ]:
t,y1 = plot_sin_curve(a = 1.5, f = 50, pd = 0, end_time = 1, sample_num = 1024)

In [ ]:
t,y2 = plot_sin_curve(a = 1, f = 120, pd = np.pi/4*0, end_time = 1, sample_num = 1024)

In [ ]:
y = y1 + y2

plt.figure(figsize=(12,8))
plt.grid()
plt.plot(t,y)

In [ ]:
def plot_fft(y:np.ndarray, t:np.ndarray, fs:int, sample_num:int, value_name:str):
    """Generate fft and plot

    Args:
        y (np.ndarray): time series value.
        t (np.ndarray): time.
        fs(int):sampling frequency.
        sample_num(int): sample number.
        value_name(str): name of value
    """
    
    sp = np.fft.fft(y)
    freq = np.fft.fftfreq(t.shape[-1], d=1.0/fs)

#     fig, ax = plt.subplots(nrows=3, sharex=True, figsize=(12,12))
#     ax[0].plot(sp.real, label="Real part")
#     ax[0].legend()
#     ax[1].plot(sp.imag, label="Imaginary part")
#     ax[1].legend()
#     ax[2].plot(freq, label="Frequency")
#     ax[2].legend()
#     ax[2].set_xlabel("Number of data")

#     plt.grid()
#     plt.show()
    
    Amp = np.abs(sp/(sample_num/2)) # Amplitude

    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(freq[1:int(sample_num/2)], Amp[1:int(sample_num/2)])
    plt.title(f"FFT of {value_name}",fontsize=30)
    ax.set_xlabel("Freqency [Hz]")
    ax.set_ylabel("Amplitude")
    ax.grid()
    #plt.xlim(0,5)#Change range according to frequency of wave
    plt.show()

In [ ]:
plot_fft(y=y,t=t,fs=1000,sample_num=1024,value_name="sin wave")

## Example2: White noize

In [ ]:
import numpy as np
dlen = 1024 #ノイズデータのデータ長
mean = 0.0  #ノイズの平均値
std  = 1.0  #ノイズの分散

y = np.random.normal(mean,std,dlen)

In [ ]:
def plot_white_noize(mean:float, std:float, end_time:float, sample_num:int):
    """Generate sin curve and plot

    Args:
        mean (float): average.
        std(float):standard deviation.
        end_time (float): End time(sec).
        sample_num(int) sample number.
    
    Returns:
        t(np.ndarray): numpy array of time
        y(np.ndarray):numpy array of y
    """
    t = np.linspace(0, end_time, sample_num)
    y = np.random.normal(mean,std,sample_num)
    plt.figure(figsize=(12,8))
    plt.plot(t,y)
    plt.grid()
    plt.show()
    
    return t,y

In [ ]:
t, y = plot_white_noize(mean=0, std=1.0, end_time=10, sample_num=1024)

In [ ]:
plot_fft(y=y,t=t,fs=1000,sample_num=1024,value_name="White noize")

# FFT for cryptocurrency

In [ ]:
for asset_id in asset_df["Asset_ID"].values:
    value_name = asset_df[asset_df["Asset_ID"]==asset_id]["Asset_Name"].values[0]
    t, y = plot_time_series(asset_id = asset_id, asset_value = "Target",start = 0, end = 1024, visualise=False)
    plot_fft(y=y,t=t,fs=1000,sample_num=1024,value_name=value_name)

# Lomb-Scargle periodogram

For details of Lomb-Scargle periodogram, please check below reference.

**Caution**  
For now, this does not work properly, if "drop_na = False", it does not output any...  
So it is set as True.

In [ ]:
import scipy.signal as signal

asset_id = 8
value_name = asset_df[asset_df["Asset_ID"]==asset_id]["Asset_Name"].values[0]
t, y = plot_time_series(asset_id = asset_id,asset_value = "Target",start = 0, end = 1024, visualise=False,drop_na = True )

f = np.linspace(0.01, 100, 10000)
pgram = signal.lombscargle(t, y, f, normalize=True)
plt.plot(f, pgram)
plt.show()

# Reference
1. [numpy.fft.fft](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html)
2. [Spectral Analysis of Nonuniformly Sampled Signals](https://www.mathworks.com/help/signal/ug/spectral-analysis-of-nonuniformly-sampled-signals.html)
3. [scipy.signal.lombscargle](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lombscargle.html)